In [20]:
!pip install keras-tuner
import keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [22]:
print(tf.__version__)

2.8.2


In [23]:
fashion_mnist = keras.datasets.fashion_mnist

In [24]:
(train_images,train_labels),(test_images,test_labels)= fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [25]:
train_images=train_images/255.0
test_images=test_images/255.0

In [26]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [27]:
train_images= train_images.reshape(len(train_images),28,28,1)
test_images= test_images.reshape(len(test_images),28,28,1)

In [28]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']) 
  return model
  

In [29]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [30]:
tuner_search = RandomSearch(build_model,objective = 'val_accuracy',max_trials=5,directory = 'output',project_name= 'Mnist Fashion')

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json


INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json


In [32]:
tuner_search.search(train_images,train_labels,epochs = 3,validation_split=0.1)

Trial 5 Complete [00h 00m 30s]
val_accuracy: 0.8771666884422302

Best val_accuracy So Far: 0.9104999899864197
Total elapsed time: 00h 03m 59s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [33]:
model = tuner_search.get_best_models(num_models=1)[0]

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 112)       2912      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        48432     
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 80)                1858640   
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
Total params: 1,910,794
Trainable params: 1,910,794
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch = 3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1614 - accuracy: 0.9391 - val_loss: 0.2646 - val_accuracy: 0.9073
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1255 - accuracy: 0.9523 - val_loss: 0.2969 - val_accuracy: 0.9080
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0941 - accuracy: 0.9655 - val_loss: 0.3093 - val_accuracy: 0.9098
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0733 - accuracy: 0.9726 - val_loss: 0.3647 - val_accuracy: 0.9100
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0578 - accuracy: 0.9782 - val_loss: 0.3858 - val_accuracy: 0.9147
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0449 - accuracy: 0.9837 - val_loss: 0.4438 - val_accuracy: 0.9068
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0381 - accuracy: 0.9865 - val_loss: 0.4934 - val_accuracy